In [52]:
peptidesPTM = []
with open('./data/input.tsv') as f:
    for line in f:
        if 'UniMod' in line:
            peptidesPTM.append(line)

In [53]:
peptidesPTM[:5]

['DN(UniMod:5)SVHWERPQKPK\n',
 'DVFFG(UniMod:6)PK\n',
 'FKDLGEEN(UniMod:5)FK\n',
 'IQHC(UniMod:4)KKLSSWVLLM(UniMod:35)K\n',
 'L(UniMod:7)AEYHAKATEHLSTLSEK\n']

In [57]:
peptidesPTM = [peptide.rstrip() for peptide in peptidesPTM]

In [58]:
peptidesPTM[:5]

['DN(UniMod:5)SVHWERPQKPK',
 'DVFFG(UniMod:6)PK',
 'FKDLGEEN(UniMod:5)FK',
 'IQHC(UniMod:4)KKLSSWVLLM(UniMod:35)K',
 'L(UniMod:7)AEYHAKATEHLSTLSEK']

In [79]:
test_peptide = peptidesPTM[3]

In [102]:
test_peptide.find('HC')

2

In [100]:
search_strs = test_peptide.split('(')
for idx, search_str in enumerate(search_strs):
    if 'Uni' in search_str:
        close_loc = search_str.find(')')
        search_strs[idx] = search_str[close_loc+1:]        

In [101]:
search_strs

['IQHC', 'KKLSSWVLLM', 'K']

In [63]:
proteins = {}
with open('./data/proteins.fasta') as f:
    for line in f:
        if '>' in line:
            line = line.replace('>','').rstrip()
            current_protein = line
            proteins[current_protein] = ''
        else:
            proteins[current_protein] += line.rstrip()

In [69]:
proteins['O75711'] == 'MKLMVLVFTIGLTLLLGVQAMPANRLSCYRKILKDHNCHNLPEGVADLTQIDVNVQDHFWDGKGCEMICYCNFSELLCCPKDVFFGPKISFVIPCNNQ'

True

In [229]:
for protein,sequence in proteins.items():
    if 'DVFFGPK' in sequence:
        print(f'Found in {protein}!')

Found in O75711!


In [242]:
proteins['O75711'][85]

'G'

In [231]:
proteins['O75711'].find('DVFFGPK')

81

In [233]:
this_prot_sections = proteins['O75711'].split('DVFFG')
this_prot_sections

['MKLMVLVFTIGLTLLLGVQAMPANRLSCYRKILKDHNCHNLPEGVADLTQIDVNVQDHFWDGKGCEMICYCNFSELLCCPK',
 'PKISFVIPCNNQ']

In [235]:
len(this_prot_sections[0])

81

In [240]:
this_prot_sections[0][80]

'K'

In [215]:
for section in this_prot_sections:
    print(section.find('SVHWERPQKPK')) 

-1
-1
-1


In [177]:
''.join(['h','e','l'])

'hel'

In [196]:
test_str = 'aksldjhelasdasdlkjsdhelasdjkldfg'

In [197]:
test_str.rfind('hel')

20

In [198]:
test_str[:20]

'aksldjhelasdasdlkjsd'

In [199]:
locations = []
chars = ['h','e','l']
locs_in_peptide = [0,2]
while ''.join(chars) in test_str:
    start = test_str.rfind(''.join(chars))
    for loc in locs_in_peptide:
        print(start + loc)
    test_str = test_str[:start]

20
22
6
8


In [155]:
def read_peptides(peptide_tsv_filepath):
    """
    Takes in...
    """
    peptidesPTM = []
    with open('./data/input.tsv') as f:
        for line in f:
            if 'UniMod' in line:
                peptidesPTM.append(line.rstrip())
    
    peptides_dict = {}
    for peptide in peptidesPTM:
        search_strs = peptide.split('(')
        for idx, search_str in enumerate(search_strs):
            if 'Uni' in search_str:
                close_loc = search_str.find(')')
                search_strs[idx] = search_str[close_loc+1:]
        peptides_dict.update({peptide: search_strs})
    
    return peptides_dict

In [156]:
peptides_dict = read_peptides('./data/input.tsv')

In [205]:
peptides_dict

{'DN(UniMod:5)SVHWERPQKPK': ['DN', 'SVHWERPQKPK'],
 'DVFFG(UniMod:6)PK': ['DVFFG', 'PK'],
 'FKDLGEEN(UniMod:5)FK': ['FKDLGEEN', 'FK'],
 'IQHC(UniMod:4)KKLSSWVLLM(UniMod:35)K': ['IQHC', 'KKLSSWVLLM', 'K'],
 'L(UniMod:7)AEYHAKATEHLSTLSEK': ['L', 'AEYHAKATEHLSTLSEK'],
 'L(UniMod:7)DGKFSVVYAK': ['L', 'DGKFSVVYAK'],
 'L(UniMod:7)G(UniMod:6)G(UniMod:6)HLDAK': ['L', 'G', 'G', 'HLDAK'],
 'L(UniMod:7)SKELQAAQAR': ['L', 'SKELQAAQAR'],
 'N(UniMod:5)KFDPSLTQR': ['N', 'KFDPSLTQR'],
 'AL(UniMod:7)HPEEDPEGR': ['AL', 'HPEEDPEGR'],
 'ITGKPIKL(UniMod:7)TQVEHR': ['ITGKPIKL', 'TQVEHR'],
 'IYISG(UniMod:6)MAPRPSLAKK': ['IYISG', 'MAPRPSLAKK'],
 'IHWESASL(UniMod:7)L(UniMod:7)R': ['IHWESASL', 'L', 'R'],
 'ALHPEEDPEG(UniMod:6)R': ['ALHPEEDPEG', 'R'],
 'FKDL(UniMod:7)GEENFKALVL(UniMod:7)IAFAQYLQQC(UniMod:4)PFEDHVK': ['FKDL',
  'GEENFKALVL',
  'IAFAQYLQQC',
  'PFEDHVK'],
 'ILGGHL(UniMod:7)DAK': ['ILGGHL', 'DAK'],
 'AL(UniMod:7)HPEEDPEGRQGR': ['AL', 'HPEEDPEGRQGR'],
 'ITG(UniMod:6)KPIKL(UniMod:7)TQVEHR': ['ITG', '

In [108]:
def read_proteins(protein_fasta):
    """
    Takes in...
    """
    proteins = {}
    with open('./data/proteins.fasta') as f:
        for line in f:
            if '>' in line:
                line = line.replace('>','').rstrip()
                current_protein = line
                proteins[current_protein] = ''
            else:
                proteins[current_protein] += line.rstrip()
    return proteins

In [109]:
proteins_dict = read_proteins('./data/proteins.fasta')

In [204]:
proteins_dict

{'O75711': 'MKLMVLVFTIGLTLLLGVQAMPANRLSCYRKILKDHNCHNLPEGVADLTQIDVNVQDHFWDGKGCEMICYCNFSELLCCPKDVFFGPKISFVIPCNNQ',
 'P00738': 'MSALGAVIALLLWGQLFAVDSGNDVTDIADDGCPKPPEIAHGYVEHSVRYQCKNYYKLRTEGDGVYTLNDKKQWINKAVGDKLPECEADDGCPKPPEIAHGYVEHSVRYQCKNYYKLRTEGDGVYTLNNEKQWINKAVGDKLPECEAVCGKPKNPANPVQRILGGHLDAKGSFPWQAKMVSHHNLTTGATLINEQWLLTTAKNLFLNHSENATAKDIAPTLTLYVGKKQLVEIEKVVLHPNYSQVDIGLIKLKQKVSVNERVMPICLPSKDYAEVGRVGYVSGWGRNANFKFTDHLKYVMLPVADQDQCIRHYEGSTVPEKKTPKSPVGVQPILNEHTFCAGMSKYQEDTCYGDAGSAFAVHDLEEDTWYATGILSFDKSCAVAEYGVYVKVTSIQDWVQKTIAEN',
 'P00739': 'MSDLGAVISLLLWGRQLFALYSGNDVTDISDDRFPKPPEIANGYVEHLFRYQCKNYYRLRTEGDGVYTLNDKKQWINKAVGDKLPECEAVCGKPKNPANPVQRILGGHLDAKGSFPWQAKMVSHHNLTTGATLINEQWLLTTAKNLFLNHSENATAKDIAPTLTLYVGKKQLVEIEKVVLHPNYHQVDIGLIKLKQKVLVNERVMPICLPSKNYAEVGRVGYVSGWGQSDNFKLTDHLKYVMLPVADQYDCITHYEGSTCPKWKAPKSPVGVQPILNEHTFCVGMSKYQEDTCYGDAGSAFAVHDLEEDTWYAAGILSFDKSCAVAEYGVYVKVTSIQHWVQKTIAEN',
 'P01008': 'MYSNVIGTVTSGKRKVYLLSLLLIGFWDCVTCHGSPVDICTAKPRDIPMNPMCIYRSPEKKATEDEGSEQKIPEATNRRVWELSKANSRFA

In [243]:
def get_PTM_info(peptides_dict,proteins_dict):
    """
    Takes in...
    """
    PTM_info_list = []
    for protein,sequence in proteins_dict.items():
        for peptide,search_strs in peptides_dict.items():
            mod_idx = -1
            locations = []
            for s in search_strs[:-1]:
                mod_idx += len(s)
                locations.append(mod_idx)
            sequence_idx = []
            full_pep = ''.join(search_strs)
            while full_pep in sequence:
                print(f'Found {peptide} in {protein}!')
                start = sequence.rfind(full_pep)
                for loc in locations:
                    sequence_idx.append(start + loc)
                sequence = sequence[:start]
            if sequence_idx != []:
                PTM_info_list.append([protein,peptide,sequence_idx])
            
    return PTM_info_list

In [244]:
info_list = get_PTM_info(peptides_dict,proteins_dict)

Found DVFFG(UniMod:6)PK in O75711!
Found L(UniMod:7)G(UniMod:6)G(UniMod:6)HLDAK in P00738!
Found L(UniMod:7)G(UniMod:6)G(UniMod:6)HLDAK in P00739!
Found FRIEDGFSL(UniMod:7)K in P01008!
Found IQHC(UniMod:4)KKLSSWVLLM(UniMod:35)K in P01009!
Found DN(UniMod:5)SVHWERPQKPK in P01023!
Found IHWESASL(UniMod:7)L(UniMod:7)R in P01024!
Found L(UniMod:7)AEYHAKATEHLSTLSEK in P02647!
Found L(UniMod:7)SKELQAAQAR in P02649!
Found L(UniMod:7)DGKFSVVYAK in P02765!
Found FKDLGEEN(UniMod:5)FK in P02768!
Found IYISG(UniMod:6)MAPRPSLAKK in P04004!
Found IIYKEN(UniMod:5)ERFQYK in P08603!
Found N(UniMod:5)KFDPSLTQR in P08697!
Found ITGKPIKL(UniMod:7)TQVEHR in P36955!
Found IHL(UniMod:7)MAGR in Q5T013!
Found AL(UniMod:7)HPEEDPEGR in Q96GW7!


In [245]:
info_list

[['O75711', 'DVFFG(UniMod:6)PK', [85]],
 ['P00738', 'L(UniMod:7)G(UniMod:6)G(UniMod:6)HLDAK', [162, 163, 164]],
 ['P00739', 'L(UniMod:7)G(UniMod:6)G(UniMod:6)HLDAK', [104, 105, 106]],
 ['P01008', 'FRIEDGFSL(UniMod:7)K', [362]],
 ['P01009', 'IQHC(UniMod:4)KKLSSWVLLM(UniMod:35)K', [255, 265]],
 ['P01023', 'DN(UniMod:5)SVHWERPQKPK', [1178]],
 ['P01024', 'IHWESASL(UniMod:7)L(UniMod:7)R', [1317, 1318]],
 ['P02647', 'L(UniMod:7)AEYHAKATEHLSTLSEK', [212]],
 ['P02649', 'L(UniMod:7)SKELQAAQAR', [110]],
 ['P02765', 'L(UniMod:7)DGKFSVVYAK', [120]],
 ['P02768', 'FKDLGEEN(UniMod:5)FK', [41]],
 ['P04004', 'IYISG(UniMod:6)MAPRPSLAKK', [357]],
 ['P08603', 'IIYKEN(UniMod:5)ERFQYK', [229]],
 ['P08697', 'N(UniMod:5)KFDPSLTQR', [242]],
 ['P36955', 'ITGKPIKL(UniMod:7)TQVEHR', [352]],
 ['Q5T013', 'IHL(UniMod:7)MAGR', [106]],
 ['Q96GW7', 'AL(UniMod:7)HPEEDPEGR', [879]]]

In [246]:
proteins_dict['P00739']

'MSDLGAVISLLLWGRQLFALYSGNDVTDISDDRFPKPPEIANGYVEHLFRYQCKNYYRLRTEGDGVYTLNDKKQWINKAVGDKLPECEAVCGKPKNPANPVQRILGGHLDAKGSFPWQAKMVSHHNLTTGATLINEQWLLTTAKNLFLNHSENATAKDIAPTLTLYVGKKQLVEIEKVVLHPNYHQVDIGLIKLKQKVLVNERVMPICLPSKNYAEVGRVGYVSGWGQSDNFKLTDHLKYVMLPVADQYDCITHYEGSTCPKWKAPKSPVGVQPILNEHTFCVGMSKYQEDTCYGDAGSAFAVHDLEEDTWYAAGILSFDKSCAVAEYGVYVKVTSIQHWVQKTIAEN'

In [248]:
proteins_dict['P00739'][104]

'L'

In [249]:
proteins_dict['P00739'][105]

'G'

In [250]:
proteins_dict['P00739'][106]

'G'

In [251]:
proteins_dict['P01009'][255]

'C'

In [252]:
proteins_dict['P01009'][265]

'M'

In [269]:
def write_to_csv(PTM_info_list,output_file_path):
    """
    Takes in...
    """
    with open(output_file_path,mode = 'w') as f:
        f.write('ProteinName,FullPeptideName,ModifiedAminoPositions\n')
        for info in PTM_info_list:
            idx_as_strs = [ str(num) for num in info[2]]
            locations = ';'.join(idx_as_strs)
            f.write(f'{info[0]},{info[1]},{locations}\n')
            
    return

In [270]:
write_to_csv(info_list,'./data/output.csv')

In [278]:
def find_PTMs(peptide_tsv_filepath,protein_fasta_filepath,output_filepath):
    """
    Takes in a...
    """
    peptides_dict = read_peptides(peptide_tsv_filepath)
    proteins_dict = read_proteins(protein_fasta_filepath)
    
    PTM_info_list = get_PTM_info(peptides_dict,proteins_dict)
    
    write_to_csv(PTM_info_list,output_filepath)
    
    print(f'Saved file to {output_filepath}')
    
    return            

In [279]:
find_PTMs('./data/input.tsv','./data/proteins.fasta','./data/output.csv')

Found DVFFG(UniMod:6)PK in O75711!
Found L(UniMod:7)G(UniMod:6)G(UniMod:6)HLDAK in P00738!
Found L(UniMod:7)G(UniMod:6)G(UniMod:6)HLDAK in P00739!
Found FRIEDGFSL(UniMod:7)K in P01008!
Found IQHC(UniMod:4)KKLSSWVLLM(UniMod:35)K in P01009!
Found DN(UniMod:5)SVHWERPQKPK in P01023!
Found IHWESASL(UniMod:7)L(UniMod:7)R in P01024!
Found L(UniMod:7)AEYHAKATEHLSTLSEK in P02647!
Found L(UniMod:7)SKELQAAQAR in P02649!
Found L(UniMod:7)DGKFSVVYAK in P02765!
Found FKDLGEEN(UniMod:5)FK in P02768!
Found IYISG(UniMod:6)MAPRPSLAKK in P04004!
Found IIYKEN(UniMod:5)ERFQYK in P08603!
Found N(UniMod:5)KFDPSLTQR in P08697!
Found ITGKPIKL(UniMod:7)TQVEHR in P36955!
Found IHL(UniMod:7)MAGR in Q5T013!
Found AL(UniMod:7)HPEEDPEGR in Q96GW7!
Saved file to ./data/output.csv
